In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


In [2]:
#load data into pandas dataframe
url = 'https://raw.githubusercontent.com/scrannad/Medicare-Project/annas_branch/numeric_only.csv'
df = pd.read_csv(url)
df.head(25)

,Beneficiaries_with_A_and_B,Fee_for_service_beneficiaries,Med_Advantage_Count,Med_Advantage_Rate,Average_Age,%Female,%Male,%White,%Black,%Hispanic,...,Dialysis_User_Count,Dialysis_User_%,Dialysis_Visits_Per_1000_Users,FQHC_Std_Costs,FQHC_Std_As_%Total,FQHC_Std_Per_Capita,FQHC_Std_Per_User,FQHC_User_Count,FQHC_User_%,FQHC_Visits_Per_1000_Users
0,6444,6186,258,0.0400,71,0.5144,0.4856,0.8810,0.0063,0.0833,...,40,0.0065,741,330449.22,0.0068,53.42,306.26,1079,0.1744,553
1,7463,6861,602,0.0807,73,0.5270,0.4730,0.9469,0.0020,0.0144,...,19,0.0028,437,37418.97,0.0007,5.45,558.49,67,0.0098,39
2,14583,13311,1272,0.0872,71,0.5330,0.4670,0.9246,0.0072,0.0336,...,74,0.0056,665,1257134.97,0.0100,94.44,554.54,2267,0.1703,773
3,18984,17907,1077,0.0567,72,0.5456,0.4544,0.8645,0.0168,0.0782,...,86,0.0048,554,1058899.69,0.0063,59.13,605.43,1749,0.0977,387
4,6134,5925,209,0.0341,70,0.5232,0.4768,0.9391,0.0025,0.0253,...,38,0.0064,659,69716.25,0.0013,11.77,400.67,174,0.0294,86
5,5256,4734,522,0.0993,72,0.5239,0.4761,0.8925,0.0055,0.0606,...,18,0.0038,481,111911.30,0.0027,23.64,592.12,189,0.0399,170
6,100102,93339,6763,0.0676,74,0.5273,0.4727,0.9182,0.0040,0.0373,...,247,0.0026,327,10000026.42,0.0122,107.14,625.00,16000,0.1714,652
7,11777,10945,832,0.0706,52,0.4838,0.5162,0.8429,0.0170,0.0744,...,209,0.0191,2238,1870255.00,0.0156,170.88,704.69,2654,0.2425,1164
8,111879,104284,7595,0.0679,72,0.5227,0.4773,0.9103,0.0053,0.0412,...,456,0.0044,527,11870281.41,0.0127,113.83,636.34,18654,0.1789,706
9,6065,2703,3362,0.5543,67,0.4639,0.5361,0.9811,0.0070,0.0041,...,29,0.0107,1118,1023021.51,0.0388,378.48,596.86,1714,0.6341,4074


In [3]:
#Check out data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17749 entries, 0 to 17748
Data columns (total 82 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Beneficiaries_with_A_and_B                   17749 non-null  int64  
 1   Fee_for_service_beneficiaries                17749 non-null  int64  
 2   Med_Advantage_Count                          17749 non-null  int64  
 3   Med_Advantage_Rate                           17749 non-null  float64
 4   Average_Age                                  17749 non-null  int64  
 5   %Female                                      17749 non-null  float64
 6   %Male                                        17749 non-null  float64
 7   %White                                       17749 non-null  float64
 8   %Black                                       17749 non-null  float64
 9   %Hispanic                                    17749 non-null  float64
 10

In [13]:
readmit_count=df['Hospital_Readmit_Rate%'].value_counts()
readmit_count

0.1703    51
0.1667    48
0.1767    46
0.1776    43
0.1720    43
          ..
0.0936     1
0.2480     1
0.2482     1
0.2769     1
0.2288     1
Name: Hospital_Readmit_Rate%, Length: 1427, dtype: int64

In [4]:
# Split preprocessed data into features and target arrays
X=df.drop(['Hospital_Readmit_Rate%'], axis=1).values
y=df['Hospital_Readmit_Rate%'].values

# Split preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test=train_test_split(X,y, random_state=3)

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Get the shape
X_train_scaled.shape

(13311, 81)

In [7]:
input = X_train_scaled.shape[1]

In [8]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 11.6 MB/s eta 0:00:00


In [9]:
#create function to allow kerastuner to decide which activation functions, neurons, and hidden layers optimize output
def create_model(hp):
    nn = tf.keras.models.Sequential()

    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 90,
        step=5), activation=activation, input_dim=input))

    for i in range(hp.Int('num_layers', 1, 5)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn

In [10]:
#add tuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [11]:
# Run the kerastuner 
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 137 Complete [00h 00m 14s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 14m 00s

Search: Running Trial #138

Value             |Best Value So Far |Hyperparameter
relu              |tanh              |activation
66                |86                |first_units
2                 |3                 |num_layers
26                |6                 |units_0
1                 |1                 |units_1
1                 |1                 |units_2
6                 |None              |units_3
26                |None              |units_4
12                |2                 |tuner/epochs
4                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
2                 |0                 |tuner/round
0104              |None              |tuner/trial_id

Epoch 5/12
416/416 [==============================] - 2s 3ms/step - loss: 0.5053 - accuracy: 0.0000e+00 - val_loss: 0.4904 - val_accuracy: 0.0000e+00
Epoch 6/12


KeyboardInterrupt: ignored